In [54]:
import pandas
import numpy
from scipy.signal import spectrogram
from scipy import signal
import miniaudio
from matplotlib.pyplot import specgram
import soundfile as sf  
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio
import numpy as np
import sklearn
import os
import glob
import pandas as pd
import pickle

from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [145]:
mfccs = librosa.feature.mfcc(a, n_mfcc=40)
mfccs.shape

(40, 94)

In [33]:
# Get audio files for 2 speakers
path = '/Users/greenapple/project5/data/raw/LibriSpeech/train-clean-360' # folder with training data
files = files_for_modeling(path, 2)

In [34]:
# Number of files for the first 2 speakers
len(files)

232

In [35]:
# Functions - take out

def files_for_modeling(path, speaker_num):

    path = '/Users/greenapple/project5/data/raw/LibriSpeech/dev-clean'

    id_folder_list =  glob.glob(os.path.join(path, '*'))
    # folders_ids =  glob.glob(os.path.join(data_dir, '*'))
    files = []
#     speaker_num = len(id_folder_list)
    id_folder_list = id_folder_list[:speaker_num]
    
    for id_folder in id_folder_list:
        file_path  = glob.glob(os.path.join(id_folder, '*', '*.flac'))
        for file in file_path:      
            if os.stat(file).st_size > 75000:
                files.append(file)
           
    return files


def load_audio_file(file, sample_rate=16000, offset=0.4, duration=3):
    
    audio, sample_rate = librosa.load(file, sr=sample_rate, offset=offset, duration=duration)
    
    return audio, sample_rate

def fourier_transform(x):
    X = librosa.stft(x, n_fft=512, hop_length=200)
    Xdb = librosa.amplitude_to_db(abs(X))
    return Xdb


def pre_processing(Xdb):    
    Xdb = sklearn.preprocessing.scale(Xdb, axis=1, copy=False) # Scale
    Xdb_resized = np.resize(Xdb, (224, 224))
    Xdb_3D = np.stack((Xdb_resized, Xdb_resized, Xdb_resized),axis = 2)
    return Xdb_3D


def speaker_id(file):
    path_split_1 = os.path.split(file)
    path_split_2 = os.path.split(path_split_1[0])
    path_split_3 = os.path.split(path_split_2[0])  
    y = int(path_split_3[1])
    return y


def one_observation(file):
    audio, _ = load_audio_file(file)
    Xdb = fourier_transform(audio)
    Xdb_3D = pre_processing(Xdb)
    y = speaker_id(file)
    
    return Xdb_3D, y

In [36]:
# Process features
data = pd.DataFrame()
Xdb_3D_list = []
y_list = []

for file in files:
    Xdb_3D, y = one_observation(file) # Features and label for one obervation = audio file
    
    Xdb_3D_list.append(Xdb_3D)
    y_list.append(y)
    
data['y'] = y_list
data['features'] = Xdb_3D_list     

/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. 

/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical is

/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical is

/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical is

/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical is

...

/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical is

/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical is

/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical is

/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical is

/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/Users/greenapple/anaconda3/envs/project4/lib/python3.7/site-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical is

In [38]:
X = data['features']
y = data['y']

In [39]:
X.shape, y.shape

((232,), (232,))

In [40]:
data.columns

Index(['y', 'features'], dtype='object')

In [41]:
len(set(data.y.values))

2

In [42]:
X = np.array(data.features.tolist())
X.shape

(232, 224, 224, 3)

In [43]:
y = np.array(data.y.tolist())
y.shape

(232,)

In [44]:
y = to_categorical(LabelEncoder().fit_transform(y))

In [45]:
y.shape

(232, 2)

In [46]:
from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Activation

# note we exclude the final dense layers and add one back below, we would retrain it ourselves
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224, 3)) 
 
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    
    
x = base_model.output
x = Flatten()(x) # flatten from convolution tensor output 
predictions = Dense(2, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions, )

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
            loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [47]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [48]:
model.fit(X, y, epochs=10, verbose=True, batch_size=16, validation_split=0.2)

Train on 185 samples, validate on 47 samples
Epoch 1/10
185/185 [==============================] - 112s 603ms/step - loss: 0.6757 - acc: 0.5622 - val_loss: 1.1734 - val_acc: 0.0000e+00
Epoch 2/10
185/185 [==============================] - 121s 654ms/step - loss: 0.6608 - acc: 0.6270 - val_loss: 0.6866 - val_acc: 0.5532
Epoch 3/10
185/185 [==============================] - 124s 669ms/step - loss: 0.5991 - acc: 0.6378 - val_loss: 0.5404 - val_acc: 0.9149
Epoch 4/10
185/185 [==============================] - 128s 693ms/step - loss: 0.5536 - acc: 0.6973 - val_loss: 0.5432 - val_acc: 0.8936
Epoch 5/10
185/185 [==============================] - 124s 670ms/step - loss: 0.5319 - acc: 0.7405 - val_loss: 0.4946 - val_acc: 0.9149
Epoch 6/10
185/185 [==============================] - 127s 688ms/step - loss: 0.5010 - acc: 0.7243 - val_loss: 0.3461 - val_acc: 0.9787
Epoch 7/10
185/185 [==============================] - 114s 618ms/step - loss: 0.5211 - acc: 0.7405 - val_loss: 0.1807 - val_acc: 1.0000

In [ ]:
# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience=2)

In [52]:
model.fit(X, y, epochs=10, verbose=True, batch_size=16, validation_split=0.2, 
          callbacks = [early_stopping_monitor])

Train on 185 samples, validate on 47 samples
Epoch 1/10
185/185 [==============================] - 112s 607ms/step - loss: 0.4036 - acc: 0.8595 - val_loss: 0.8125 - val_acc: 0.3617
Epoch 2/10
185/185 [==============================] - 117s 634ms/step - loss: 0.3828 - acc: 0.8703 - val_loss: 0.8514 - val_acc: 0.3404
Epoch 3/10
185/185 [==============================] - 119s 645ms/step - loss: 0.3579 - acc: 0.8811 - val_loss: 0.8062 - val_acc: 0.4255
Epoch 4/10
185/185 [==============================] - 119s 645ms/step - loss: 0.3522 - acc: 0.8432 - val_loss: 0.4266 - val_acc: 0.8723
Epoch 5/10
185/185 [==============================] - 116s 625ms/step - loss: 0.3442 - acc: 0.8649 - val_loss: 0.5730 - val_acc: 0.7872
Epoch 6/10
185/185 [==============================] - 126s 682ms/step - loss: 0.3346 - acc: 0.9189 - val_loss: 0.5672 - val_acc: 0.7872


In [53]:
model.fit(X, y, epochs=10, verbose=True, batch_size=16, validation_split=0.2, 
          callbacks = [early_stopping_monitor])

Train on 185 samples, validate on 47 samples
Epoch 1/10
185/185 [==============================] - 113s 611ms/step - loss: 0.3248 - acc: 0.8811 - val_loss: 0.3161 - val_acc: 0.9149
Epoch 2/10
185/185 [==============================] - 126s 681ms/step - loss: 0.3195 - acc: 0.9081 - val_loss: 0.3308 - val_acc: 0.9149
Epoch 3/10
185/185 [==============================] - 127s 686ms/step - loss: 0.3216 - acc: 0.9135 - val_loss: 0.4196 - val_acc: 0.8723


In [56]:
# Pickel model
pickle_in = open('/Users/greenapple/project5/models/01_VGG16_2_cls.pkl', 'wb')
pickle.dump(model, pickle_in)